In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller, kpss
from scipy.stats import linregress
import os
import warnings
from typing import List, Dict, Tuple, Optional, Union, Callable
from statsmodels.tools.sm_exceptions import InterpolationWarning

class TimeSeriesAnalyzer:
    
    def __init__(self, file_path: str, close_column: str = 'close', date_column: Optional[str] = None):
        self.file_path = file_path
        self.close_column = close_column
        self.date_column = date_column 
        self.data = None
        self.results = {}  
        
    def load_data(self, drop_na: bool = True) -> None:
        try:
            self.data = pd.read_csv(self.file_path)
            if self.close_column not in self.data.columns:
                raise ValueError(f"数据中不存在名为'{self.close_column}'的列")
                
            #### 处理缺失值
            if self.date_column and self.date_column in self.data.columns:
                self.data[self.date_column] = pd.to_datetime(self.data[self.date_column])
                
            if drop_na:
                self.data = self.data.dropna(subset=[self.close_column])
                print(f"删除缺失值后，剩余{len(self.data)}行数据")
                
            print(f"成功加载数据，共{len(self.data)}行")
        except Exception as e:
            print(f"加载数据失败: {e}")
            raise
    
    def calculate_hurst(self, series: pd.Series, min_lag: int = 10, max_lag: int = 100) -> float:
        
        ### 确保max_lag不超过序列长度的一半
        max_lag = min(max_lag, len(series) // 2)
        
        ### 检查是否有足够的数据点
        if max_lag < min_lag:
            print(f"警告: 序列长度({len(series)})过短，无法计算Hurst指数，返回NaN")
            return np.nan
            
        ### 对数收益率
        log_returns = np.log(series).diff().dropna()
        
        ### 检查对数收益率是否有足够的数据点
        if len(log_returns) <= max_lag:
            print(f"警告: 对数收益率序列长度不足，返回NaN")
            return np.nan
            
        lags = range(min_lag, max_lag + 1)
        rs_values = []
        
        for lag in lags:
            ###  计算R/S统计量
            rs = []
            for i in range(len(log_returns) - lag + 1):
                segment = log_returns.iloc[i:i+lag]
                deviations = segment - segment.mean()
                cumulative = deviations.cumsum()
                range_ = cumulative.max() - cumulative.min()
                std_ = segment.std()
                if std_ != 0:    ##### 避免除以0
                    rs.append(range_ / std_)
             ##### 如果没有有效的R/S值，跳过这个滞后       
            if not rs:
                continue
            rs_values.append(np.mean(rs))
        
        ##### 检查是否有足够的R/S值进行回归
        if len(rs_values) < 3:   ##### 至少需要3个点进行线性回归
            print(f"警告: R/S值不足，返回NaN")
            return np.nan
            
        ####### 线性回归计算Hurst
        try:
            with warnings.catch_warnings():
                warnings.filterwarnings('error')  
                slope, _, _, _, _ = linregress(np.log(lags), np.log(rs_values))
                return slope
        except Exception as e:
            print(f"计算Hurst指数错误: {e}，返回NaN")
            return np.nan
    
    def run_adf_test(self, series: pd.Series) -> Dict[str, Union[float, int]]:
        
        ##### 处理可能的缺失值
        series = series.dropna()
        result = adfuller(series, autolag='AIC')
        return {
            'statistic': result[0],
            'pvalue': result[1],
            'lags': result[2],
            'critical_values': result[4]
        }
    
    def run_kpss_test(self, series: pd.Series, regression: str = 'ct', suppress_warnings: bool = True) -> Dict[str, Union[float, Dict[str, float]]]:
        
        ##### 处理可能的缺失值
        series = series.dropna()
        
        if suppress_warnings:
            with warnings.catch_warnings(record=True) as w:
                warnings.simplefilter("always")
                result = kpss(series, regression=regression)
                
                warning_msg = None
                for warning in w:
                    if isinstance(warning.message, InterpolationWarning):
                        warning_msg = str(warning.message)
        else:
            result = kpss(series, regression=regression)
            warning_msg = None
            
        return {
            'statistic': result[0],
            'pvalue': result[1],
            'critical_values': result[3],
            'warning': warning_msg
        }
    
    def analyze_windows(self, windows: List[int], 
                        step: int = 1,  ##### 滑动步长
                        min_lag_func: Callable[[int], int] = lambda w: max(5, w // 6),
                        max_lag_func: Callable[[int], int] = lambda w: max(10, w // 3),
                        drop_na: bool = True, 
                        suppress_warnings: bool = True) -> None:
        #### 滑动窗口逻辑
        if self.data is None:
            self.load_data(drop_na=drop_na)
            
        close_series = self.data[self.close_column]
        total_data = len(close_series)
        
        for window_size in windows:
            ##### 结果列表
            self.results[window_size] = []
            
            ##### 计算滞后参数
            min_lag = min_lag_func(window_size)
            max_lag = max_lag_func(window_size)
            window_max_lag = min(max_lag, window_size // 2)
            
            ###### 检查窗口大小是否有效
            if window_max_lag < min_lag:
                print(f"窗口大小({window_size})过小，无法使用min_lag={min_lag}进行分析，跳过")
                continue
                
            ######  检查是否有足够数据
            if total_data < window_size:
                print(f"数据量({total_data})小于窗口大小({window_size})，无法分析，跳过")
                continue
                
            ######  计算滑动窗口数量
            num_windows = (total_data - window_size) // step + 1
            print(f"\n开始分析窗口大小: {window_size}，步长: {step}，总滑动窗口数: {num_windows}")
            
            ###### 滑动窗口循环
            for i in range(0, total_data - window_size + 1, step):
                start_idx = i
                end_idx = i + window_size - 1
                
                # 打印进度
                if i % (step * 10) == 0:
                    progress = (i // step) / num_windows * 100
                    print(f"进度: {progress:.1f}% ({i//step + 1}/{num_windows})")
                
                try:
                    ######  提取当前滑动窗口数据
                    window_data = close_series.iloc[start_idx:end_idx+1]
                    
                    ######  获取时间范围
                    time_range = None
                    if self.date_column and self.date_column in self.data.columns:
                        start_time = self.data.iloc[start_idx][self.date_column]
                        end_time = self.data.iloc[end_idx][self.date_column]
                        time_range = f"{start_time} 至 {end_time}"
                    
                    ######  计算3个指标
                    hurst = self.calculate_hurst(window_data, min_lag, window_max_lag)
                    adf_result = self.run_adf_test(window_data)
                    kpss_result = self.run_kpss_test(window_data, regression='ct', 
                                                    suppress_warnings=suppress_warnings)
                    
                    ######  保存结果
                    self.results[window_size].append({
                        'window_id': i // step + 1,
                        'start_idx': start_idx,
                        'end_idx': end_idx,
                        'time_range': time_range,
                        'hurst': hurst,
                        'adf': adf_result,
                        'kpss': kpss_result,
                        'min_lag': min_lag,
                        'window_max_lag': window_max_lag,
                        'trend_score': self._calculate_trend_score({
                            'hurst': hurst, 'adf': adf_result, 'kpss': kpss_result
                        }),
                        'trend_type': self._classify_trend_type({
                            'hurst': hurst, 'adf': adf_result, 'kpss': kpss_result
                        })
                    })
                    
                except Exception as e:
                    print(f"分析窗口 {i//step + 1} 时出错: {e}")
                    continue
            
            print(f"窗口大小 {window_size} 分析完成，有效窗口数: {len(self.results[window_size])}")
    
    def _calculate_trend_score(self, res: Dict) -> int:

        if np.isnan(res['hurst']):
            return 0
            
        score = 0
        
        if res['hurst'] > 0.6:
            score += 2
        elif res['hurst'] > 0.55:
            score += 1
            
        if res['adf']['pvalue'] > 0.05:
            score += 1
            
        if res['kpss']['pvalue'] < 0.05:
            score += 1
            
        if res['hurst'] > 0.55 and res['adf']['pvalue'] > 0.05 and res['kpss']['pvalue'] < 0.05:
            score += 1
            
        return min(score, 5)
    
    def _classify_trend_type(self, res: Dict) -> str:

        if np.isnan(res['hurst']):
            return "无法确定（Hurst指数计算失败）"
            
        hurst = res['hurst']
        adf_p = res['adf']['pvalue']
        kpss_p = res['kpss']['pvalue']
        
        if hurst > 0.55 and adf_p > 0.05 and kpss_p < 0.05:
            return "强趋势且非平稳（适合趋势策略）"
        elif hurst > 0.55 and adf_p < 0.05 and kpss_p > 0.05:
            return "趋势但平稳（短期趋势可能）"
        elif hurst < 0.5 and adf_p < 0.05 and kpss_p > 0.05:
            return "震荡平稳（不适合趋势策略）"
        elif hurst > 0.55 and adf_p > 0.05 and kpss_p > 0.05:
            return "矛盾（需进一步验证）"
        else:
            return "弱趋势或反趋势"
    
    def get_results_dataframe(self) -> pd.DataFrame:

        results_df = pd.DataFrame(columns=[
            '窗口大小', '窗口编号', '起始索引', '结束索引', '时间范围',
            'Hurst指数', 'ADF p值', 'KPSS p值', 
            '趋势适配性评分', '趋势类型', 'min_lag', '实际max_lag', 'KPSS警告'
        ])
        
        for window_size, window_results in self.results.items():
            for res in window_results:
                kpss_warning = res['kpss']['warning']
                if kpss_warning:
                    if "smaller" in kpss_warning:
                        kpss_warning_simplified = "实际p值可能更小"
                    elif "greater" in kpss_warning:
                        kpss_warning_simplified = "实际p值可能更大"
                    else:
                        kpss_warning_simplified = "p值估计可能不准确"
                else:
                    kpss_warning_simplified = ""
                    
                results_df = pd.concat([results_df, pd.DataFrame({
                    '窗口大小': [window_size],
                    '窗口编号': [res['window_id']],
                    '起始索引': [res['start_idx']],
                    '结束索引': [res['end_idx']],
                    '时间范围': [res['time_range']],
                    'Hurst指数': [res['hurst']],
                    'ADF p值': [res['adf']['pvalue']],
                    'KPSS p值': [res['kpss']['pvalue']],
                    '趋势适配性评分': [res['trend_score']],
                    '趋势类型': [res['trend_type']],
                    'min_lag': [res['min_lag']],
                    '实际max_lag': [res['window_max_lag']],
                    'KPSS警告': [kpss_warning_simplified]
                })], ignore_index=True)
                
        return results_df


def analyze_time_series(
    file_path: str,
    close_column: str = 'close',
    windows: List[int] = [20, 40, 60],
    step: int = 5,  
    date_column: Optional[str] = None,  
    min_lag_func: Callable[[int], int] = lambda w: max(5, w // 6),
    max_lag_func: Callable[[int], int] = lambda w: max(10, w // 3),
    drop_na: bool = True,
    suppress_warnings: bool = True
) -> pd.DataFrame:

    analyzer = TimeSeriesAnalyzer(file_path, close_column, date_column)
    analyzer.analyze_windows(
        windows=windows,
        step=step,  
        min_lag_func=min_lag_func,
        max_lag_func=max_lag_func,
        drop_na=drop_na,
        suppress_warnings=suppress_warnings
    )
    
    results_df = analyzer.get_results_dataframe()
    
    ####### 结果表格
    try:
        from IPython.display import display
        print("\n分析结果汇总:")
        display(results_df)
    except ImportError:
        print("\n分析结果汇总:")
        print(results_df.to_csv(sep='\t', na_rep='nan'))
        
    return results_df

##################################################################################################

if __name__ == "__main__":

    file_path = "RB99_120m_2010.1.1_2025.1.31.csv"   ####### 时间轴文件或自定义轴文件
    windows = [20, 40, 60]     ###### 分析窗口大小
    step = 5     ####### 滑动步长
    
    results = analyze_time_series(
        file_path=file_path,
        windows=windows,
        step=step,  
        close_column='close',    ######## 以close进行分析
        date_column='datetime', 
        drop_na=True,
        suppress_warnings=True
    )
    
    ######  保存结果
    results.to_csv("Hurst_ADF_KPSS_results.csv", index=False)


删除缺失值后，剩余13620行数据
成功加载数据，共13620行

开始分析窗口大小: 20，步长: 5，总滑动窗口数: 2721
进度: 0.0% (1/2721)
进度: 0.4% (11/2721)
进度: 0.7% (21/2721)
进度: 1.1% (31/2721)
进度: 1.5% (41/2721)
进度: 1.8% (51/2721)
进度: 2.2% (61/2721)
进度: 2.6% (71/2721)
进度: 2.9% (81/2721)
进度: 3.3% (91/2721)
进度: 3.7% (101/2721)
进度: 4.0% (111/2721)
进度: 4.4% (121/2721)
进度: 4.8% (131/2721)
进度: 5.1% (141/2721)
进度: 5.5% (151/2721)
进度: 5.9% (161/2721)
进度: 6.2% (171/2721)
进度: 6.6% (181/2721)
进度: 7.0% (191/2721)
进度: 7.4% (201/2721)
进度: 7.7% (211/2721)
进度: 8.1% (221/2721)
进度: 8.5% (231/2721)
进度: 8.8% (241/2721)
进度: 9.2% (251/2721)
进度: 9.6% (261/2721)
进度: 9.9% (271/2721)
进度: 10.3% (281/2721)
进度: 10.7% (291/2721)
进度: 11.0% (301/2721)
进度: 11.4% (311/2721)
进度: 11.8% (321/2721)
进度: 12.1% (331/2721)
进度: 12.5% (341/2721)
进度: 12.9% (351/2721)
进度: 13.2% (361/2721)
进度: 13.6% (371/2721)
进度: 14.0% (381/2721)
进度: 14.3% (391/2721)
进度: 14.7% (401/2721)
进度: 15.1% (411/2721)
进度: 15.4% (421/2721)
进度: 15.8% (431/2721)
进度: 16.2% (441/2721)
进度: 16.5% (451/2721)
进度: 16.

进度: 39.4% (1071/2717)
进度: 39.7% (1081/2717)
进度: 40.1% (1091/2717)
进度: 40.5% (1101/2717)
进度: 40.9% (1111/2717)
进度: 41.2% (1121/2717)
进度: 41.6% (1131/2717)
进度: 42.0% (1141/2717)
进度: 42.3% (1151/2717)
进度: 42.7% (1161/2717)
进度: 43.1% (1171/2717)
进度: 43.4% (1181/2717)
进度: 43.8% (1191/2717)
进度: 44.2% (1201/2717)
进度: 44.5% (1211/2717)
进度: 44.9% (1221/2717)
进度: 45.3% (1231/2717)
进度: 45.6% (1241/2717)
进度: 46.0% (1251/2717)
进度: 46.4% (1261/2717)
进度: 46.7% (1271/2717)
进度: 47.1% (1281/2717)
进度: 47.5% (1291/2717)
进度: 47.8% (1301/2717)
进度: 48.2% (1311/2717)
进度: 48.6% (1321/2717)
进度: 49.0% (1331/2717)
进度: 49.3% (1341/2717)
进度: 49.7% (1351/2717)
进度: 50.1% (1361/2717)
进度: 50.4% (1371/2717)
进度: 50.8% (1381/2717)
进度: 51.2% (1391/2717)
进度: 51.5% (1401/2717)
进度: 51.9% (1411/2717)
进度: 52.3% (1421/2717)
进度: 52.6% (1431/2717)
进度: 53.0% (1441/2717)
进度: 53.4% (1451/2717)
进度: 53.7% (1461/2717)
进度: 54.1% (1471/2717)
进度: 54.5% (1481/2717)
进度: 54.8% (1491/2717)
进度: 55.2% (1501/2717)
进度: 55.6% (1511/2717)
进度: 55.9% 

进度: 78.1% (2121/2713)
进度: 78.5% (2131/2713)
进度: 78.9% (2141/2713)
进度: 79.2% (2151/2713)
进度: 79.6% (2161/2713)
进度: 80.0% (2171/2713)
进度: 80.4% (2181/2713)
进度: 80.7% (2191/2713)
进度: 81.1% (2201/2713)
进度: 81.5% (2211/2713)
进度: 81.8% (2221/2713)
进度: 82.2% (2231/2713)
进度: 82.6% (2241/2713)
进度: 82.9% (2251/2713)
进度: 83.3% (2261/2713)
进度: 83.7% (2271/2713)
进度: 84.0% (2281/2713)
进度: 84.4% (2291/2713)
进度: 84.8% (2301/2713)
进度: 85.1% (2311/2713)
进度: 85.5% (2321/2713)
进度: 85.9% (2331/2713)
进度: 86.3% (2341/2713)
进度: 86.6% (2351/2713)
进度: 87.0% (2361/2713)
进度: 87.4% (2371/2713)
进度: 87.7% (2381/2713)
进度: 88.1% (2391/2713)
进度: 88.5% (2401/2713)
进度: 88.8% (2411/2713)
进度: 89.2% (2421/2713)
进度: 89.6% (2431/2713)
进度: 89.9% (2441/2713)
进度: 90.3% (2451/2713)
进度: 90.7% (2461/2713)
进度: 91.0% (2471/2713)
进度: 91.4% (2481/2713)
进度: 91.8% (2491/2713)
进度: 92.1% (2501/2713)
进度: 92.5% (2511/2713)
进度: 92.9% (2521/2713)
进度: 93.3% (2531/2713)
进度: 93.6% (2541/2713)
进度: 94.0% (2551/2713)
进度: 94.4% (2561/2713)
进度: 94.7% 

,窗口大小,窗口编号,起始索引,结束索引,时间范围,Hurst指数,ADF p值,KPSS p值,趋势适配性评分,趋势类型,min_lag,实际max_lag,KPSS警告
0,20,1,0,19,2010-01-04 11:00:00 至 2010-01-12 11:30:00,0.765891,1.333426e-07,0.100000,2,趋势但平稳（短期趋势可能）,5,10,实际p值可能更大
1,20,2,5,24,2010-01-05 15:00:00 至 2010-01-14 11:00:00,0.803981,9.925853e-01,0.100000,3,矛盾（需进一步验证）,5,10,实际p值可能更大
2,20,3,10,29,2010-01-07 11:30:00 至 2010-01-15 15:00:00,0.655898,1.000000e+00,0.062404,3,矛盾（需进一步验证）,5,10,
3,20,4,15,34,2010-01-11 11:00:00 至 2010-01-19 11:30:00,0.787369,3.138268e-02,0.100000,2,趋势但平稳（短期趋势可能）,5,10,实际p值可能更大
4,20,5,20,39,2010-01-12 15:00:00 至 2010-01-21 11:00:00,0.961943,2.565969e-02,0.100000,2,趋势但平稳（短期趋势可能）,5,10,实际p值可能更大
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8146,60,2709,13540,13599,2024-12-27 15:00:00 至 2025-01-20 15:00:00,0.567546,9.039810e-01,0.010000,4,强趋势且非平稳（适合趋势策略）,10,20,实际p值可能更小
8147,60,2710,13545,13604,2024-12-30 23:00:00 至 2025-01-21 23:00:00,0.574209,7.853983e-01,0.010000,4,强趋势且非平稳（适合趋势策略）,10,20,实际p值可能更小
8148,60,2711,13550,13609,2025-01-02 11:30:00 至 2025-01-23 11:00:00,0.702207,8.290949e-01,0.010000,5,强趋势且非平稳（适合趋势策略）,10,20,实际p值可能更小
8149,60,2712,13555,13614,2025-01-03 15:00:00 至 2025-01-24 11:30:00,0.703303,8.319279e-01,0.036049,5,强趋势且非平稳（适合趋势策略）,10,20,


In [2]:
###### 分析输出的 Hurst_ADF_KPSS_results.csv 文件

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from typing import List

plt.rcParams["font.family"] = ["SimHei"]
plt.rcParams["axes.unicode_minus"] = False

def analyze_trend_scores(
    result_csv: str,
    target_windows: List[int],  ##### 窗口期对应
    save_dir: str = "趋势评分分析结果"
) -> None:

    os.makedirs(save_dir, exist_ok=True)
    
    try:
        df = pd.read_csv(result_csv)
        print(f"成功读取数据，共{len(df)}条记录")
    except Exception as e:
        print(f"读取CSV失败：{e}")
        return
    
    required_cols = ["窗口大小", "窗口编号", "趋势适配性评分", "时间范围"]
    if not all(col in df.columns for col in required_cols):
        print("CSV文件缺少必要的列，请确保包含：窗口大小、窗口编号、趋势适配性评分、时间范围")
        return
    
    ###### 按窗口期分组
    valid_windows = [w for w in target_windows if w in df["窗口大小"].unique()]
    if not valid_windows:
        print(f"CSV中未找到目标窗口大小{target_windows}的数据")
        return
    print(f"将分析的窗口大小：{valid_windows}")
    
    ###### 分别可视化
    for window in valid_windows:
        window_df = df[df["窗口大小"] == window].copy()
        window_df = window_df.sort_values("窗口编号")  
        
        plt.figure(figsize=(12, 6))
        sns.lineplot(
            data=window_df,
            x="窗口编号",
            y="趋势适配性评分",
            marker="o",
            color="#2c7fb8",
            linewidth=2,
            markersize=5
        )
        
        #### 添加平均值
        mean_score = window_df["趋势适配性评分"].mean()
        plt.axhline(
            y=mean_score,
            color="#ff7f0e",
            linestyle="--",
            label=f"平均分：{mean_score:.2f}"
        )
        
        plt.title(f"窗口大小{window}的趋势适配性评分变化", fontsize=15)
        plt.xlabel("滑动窗口编号", fontsize=12)
        plt.ylabel("趋势适配性评分（0-5分）", fontsize=12)
        plt.ylim(0, 5.5)  # 评分范围固定为0-5
        plt.grid(alpha=0.3)
        plt.legend()
        plt.tight_layout()
        
        img_path = os.path.join(save_dir, f"窗口{window}_趋势评分时序图.png")
        plt.savefig(img_path, dpi=300)
        plt.close()
        print(f"已保存窗口{window}的可视化图表：{img_path}")
    
    ###### 计算对比三个窗口期的统计指标
    stats_list = []
    for window in valid_windows:
        window_df = df[df["窗口大小"] == window]
        scores = window_df["趋势适配性评分"]
        stats = {
            "窗口大小": window,
            "总窗口数": len(scores),
            "平均分": scores.mean(),
            "中位数": scores.median(),
            "最高分占比": (scores == 5).mean() * 100,  
            "低分占比": (scores <= 2).mean() * 100,  
            "评分标准差": scores.std()  
        }
        stats_list.append(stats)
    
    stats_df = pd.DataFrame(stats_list)
    stats_df = stats_df.sort_values("平均分", ascending=False)  
    stats_path = os.path.join(save_dir, "各窗口趋势评分统计.csv")
    stats_df.to_csv(stats_path, index=False)
    print(f"\n各窗口统计结果已保存：{stats_path}")
    print("\n=== 各窗口趋势适配性评分统计 ===")
    print(stats_df.round(2).to_string(index=False))  
    
    ##### 生成报告
    summary_path = os.path.join(save_dir, "分析总结.txt")
    with open(summary_path, "w", encoding="utf-8") as f:
        f.write("趋势适配性评分分析总结\n")
        f.write("="*50 + "\n")
        f.write(f"分析窗口大小：{target_windows}\n")
        f.write(f"数据来源：{result_csv}\n\n")
        
        best_window = stats_df.iloc[0]
        worst_window = stats_df.iloc[-1]
        
        f.write("1. 关键结论：\n")
        f.write(f"- 最佳窗口大小：{int(best_window['窗口大小'])}，平均评分{best_window['平均分']:.2f}\n")
        f.write(f"- 最差窗口大小：{int(worst_window['窗口大小'])}，平均评分{worst_window['平均分']:.2f}\n")
        f.write(f"- 稳定性最高：窗口{int(stats_df.loc[stats_df['评分标准差'].idxmin()]['窗口大小'])}（标准差{stats_df['评分标准差'].min():.2f}）\n")
        f.write(f"- 高趋势占比最高：窗口{int(stats_df.loc[stats_df['最高分占比'].idxmax()]['窗口大小'])}（5分占比{stats_df['最高分占比'].max():.1f}%）\n\n")
        
        f.write("2. 窗口特点分析：\n")
        for _, row in stats_df.iterrows():
            f.write(f"- 窗口{int(row['窗口大小'])}：\n")
            f.write(f"  平均分{row['平均分']:.2f}，中位数{row['中位数']:.1f}，评分波动{row['评分标准差']:.2f}\n")
            f.write(f"  高趋势窗口（5分）占比{row['最高分占比']:.1f}%，低趋势窗口（≤2分）占比{row['低分占比']:.1f}%\n")
            if row['平均分'] >= 4:
                f"  评价：该窗口大小下数据趋势性强，适合趋势策略\n"
            elif row['平均分'] >= 3:
                f"  评价：趋势性中等，部分时段适合趋势策略\n"
            else:
                f"  评价：趋势性较弱，整体不适合趋势策略\n"
    
    print(f"\n分析总结已保存：{summary_path}")


######################################################################################################

if __name__ == "__main__":
    analyze_trend_scores(
        result_csv="Hurst_ADF_KPSS_results.csv",  #####  之前保存的CSV文件
        target_windows=windows,     ##### 与之前的窗口期参数一致
        save_dir="趋势评分分析结果"  ##### 分析结果保存的目录
    )

成功读取数据，共8151条记录
将分析的窗口大小：[20, 40, 60]
已保存窗口20的可视化图表：趋势评分分析结果\窗口20_趋势评分时序图.png
已保存窗口40的可视化图表：趋势评分分析结果\窗口40_趋势评分时序图.png
已保存窗口60的可视化图表：趋势评分分析结果\窗口60_趋势评分时序图.png

各窗口统计结果已保存：趋势评分分析结果\各窗口趋势评分统计.csv

=== 各窗口趋势适配性评分统计 ===
 窗口大小  总窗口数  平均分  中位数  最高分占比  低分占比  评分标准差
   40  2717 3.51  3.0  35.59 19.21   1.26
   60  2713 3.46  3.0  37.08 29.64   1.38
   20  2721 3.06  3.0  21.46 29.58   1.23

分析总结已保存：趋势评分分析结果\分析总结.txt
